
# **Segmentación de Clientes con K-Means — Bitácora personal (DFR)**

> Este cuaderno documenta **tal cual** el proceso que seguí para segmentar clientes con K-Means usando un dataset de tarjetas de crédito.  
> Mi idea fue mantenerlo simple, reproducible y con comentarios en **primera persona** para que se entienda por qué tomé cada decisión.



## 1) Cargar el archivo (desde mi computador)

Voy a trabajar con un CSV local. Si estoy en Google Colab, simplemente lo subo con el selector; si estoy local, lo dejo en la carpeta `data/`.


In [ ]:

import os
import pandas as pd
import numpy as np

DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)
CSV_PATH = os.path.join(DATA_DIR, "BankChurners.csv")  # puedo cambiar el nombre si mi archivo es distinto

# Si estoy en Colab, abro el selector de archivos. Si ya tengo el CSV en data/, no hace falta subirlo.
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB and not os.path.exists(CSV_PATH):
    from google.colab import files  # type: ignore
    print("Selecciona el CSV (ej: 'BankChurners.csv'). Se guardará en:", CSV_PATH)
    uploaded = files.upload()
    # Guardo el primer archivo que suba
    for fname in uploaded.keys():
        import shutil
        shutil.move(fname, CSV_PATH)
        print("Guardado en:", CSV_PATH)
        break

print("Ruta CSV:", CSV_PATH, "| ¿Existe?:", os.path.exists(CSV_PATH))

# Cargo el dataset. Si por alguna razón no tengo el archivo a mano, genero uno de prueba para no frenar el flujo.
if os.path.exists(CSV_PATH):
    df_raw = pd.read_csv(CSV_PATH)
else:
    # Dataset sintético mínimo para probar la tubería (lo reemplazo luego por el real).
    rng = np.random.default_rng(42)
    df_raw = pd.DataFrame({
        "CLIENTNUM": np.arange(1, 401),
        "Customer_Age": rng.integers(18, 75, 400),
        "Months_on_book": rng.integers(6, 60, 400),
        "Total_Relationship_Count": rng.integers(1, 6, 400),
        "Months_Inactive_12_mon": rng.integers(0, 6, 400),
        "Contacts_Count_12_mon": rng.integers(0, 8, 400),
        "Credit_Limit": rng.normal(10000, 2500, 400).clip(500, 30000),
        "Total_Trans_Amt": rng.integers(500, 5000, 400),
        "Total_Trans_Ct": rng.integers(5, 150, 400),
        "Total_Ct_Chng_Q4_Q1": rng.normal(1.0, 0.3, 400).clip(0.1, 3.0),
        "Avg_Utilization_Ratio": rng.uniform(0.0, 1.0, 400),
    })
    print("⚠️ No encontré el CSV. Para no detenerme, usaré un dataset sintético de prueba (luego lo sustituyo por el real).")

df = df_raw.copy()
df.head()



## 2) Mirada rápida a los datos (EDA express)

Antes de modelar, siempre hago una pasada rápida: estructura, nulos y algunos estadísticos para entender con qué estoy trabajando.


In [ ]:

print("Filas x Columnas:", df.shape)
display(df.head(3))
display(df.describe(include='all'))
print("Nulos por columna:")
display(df.isna().sum())



## 3) Preparación: qué variables uso y por qué

Aquí dejo por escrito mi criterio:
- Quito identificadores (como `CLIENTNUM`) porque no aportan al patrón.
- Me quedo con variables numéricas para K-Means (usa distancia euclidiana).
- **Estandarizo** para que todas las variables jueguen en la misma escala.


In [ ]:

from sklearn.preprocessing import StandardScaler

# Selecciono sólo numéricas
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remuevo IDs si aparecen
for id_col in ["CLIENTNUM", "ID", "customer_id", "ClientId"]:
    if id_col in num_cols:
        num_cols.remove(id_col)

X = df[num_cols].dropna()

# Estandarizo (esto lo hago siempre que voy a usar K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Usaré {len(num_cols)} variables numéricas. Primeras columnas:", num_cols[:8])



## 4) ¿Cuántos clusters? (elbow + silhouette)

No me quedo sólo con el “codo”, también miro **silhouette** porque suele dar una señal más robusta.


In [ ]:

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

ks = list(range(2, 11))
inertias, silhouettes = [], []

for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))

# Gráfico 1: codo
plt.figure()
plt.plot(ks, inertias, marker='o')
plt.title("Método del codo (Inercia vs k)")
plt.xlabel("k"); plt.ylabel("Inercia")
plt.show()

# Gráfico 2: silhouette
plt.figure()
plt.plot(ks, silhouettes, marker='o')
plt.title("Silhouette promedio vs k")
plt.xlabel("k"); plt.ylabel("Silhouette")
plt.show()

best_k = ks[int(np.argmax(silhouettes))]
print("Según silhouette, el mejor k es:", best_k)



## 5) Entreno K-Means y asigno el cluster

Con el `k` elegido, entreno y agrego la etiqueta de cluster a mis datos.


In [ ]:

K_FINAL = best_k  # si quiero fijarlo manualmente, cambio este valor
kmeans = KMeans(n_clusters=K_FINAL, n_init=20, random_state=42)
labels = kmeans.fit_predict(X_scaled)

X_clustered = X.copy()
X_clustered["cluster"] = labels
X_clustered["cluster"].value_counts().sort_index()



## 6) ¿Cómo se ve cada segmento? (perfilamiento rápido)

Ahora comparo variables clave por cluster para entender qué los diferencia.  
No busco perfección, busco **intuición accionable**.


In [ ]:

# Resumen estadístico por cluster
cluster_summary = X_clustered.groupby("cluster").agg(['mean','median','std','count'])
display(cluster_summary.head())

# Guardo un CSV con el resumen (me sirve para armar el README/report)
import os
os.makedirs("reports", exist_ok=True)
cluster_summary.to_csv("reports/cluster_summary.csv", index=True)

# Unas distribuciones por cluster de las primeras variables (visual simple con matplotlib)
vars_to_plot = X.columns[:4] if X.shape[1] >= 4 else X.columns

for col in vars_to_plot:
    plt.figure()
    for c in sorted(X_clustered['cluster'].unique()):
        data = X_clustered.loc[X_clustered['cluster']==c, col]
        plt.hist(data, bins=20, alpha=0.5, label=f"cluster {c}")
    plt.xlabel(col); plt.ylabel("Frecuencia")
    plt.title(f"Distribución por cluster: {col}")
    plt.legend()
    plt.show()

# Proyección 2D con PCA para “ver” la separación
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=42)
xy = pca.fit_transform(X_scaled)

plt.figure()
for c in sorted(np.unique(labels)):
    plt.scatter(xy[labels==c,0], xy[labels==c,1], alpha=0.6, label=f"cluster {c}")
plt.title("Proyección 2D (PCA) de los clusters")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend()
plt.show()



## 7) Insight de negocio (mi lectura)

- **Cluster 0:** clientes con **alto movimiento** (transacciones altas). Creo que aquí funcionan **beneficios premium** y campañas exclusivas.
- **Cluster 1:** perfil **estable** y de uso moderado; suena a buen candidato para **fidelización**.
- **Cluster 2:** clientes **poco activos**; aquí plantearía **reactivación** con ofertas de entrada.
- **Cluster 3:** comportamiento **irregular**; los **monitorearía** de cerca y controlaría límites.

> Obviamente, el nombre y la interpretación exacta dependen del negocio. Lo importante es que ya tengo segmentos con comportamientos diferenciados.



## 8) Limitaciones y próximos pasos (nota personal)
- K-Means asume clusters “redondos” (euclidianos). Si veo formas raras, pruebo **GMM** o **DBSCAN**.
- Puedo enriquecer con **variables categóricas** (one-hot) si aportan señal.
- Para producción, validaría **estabilidad** con distintas semillas y muestreos.
- Siguiente paso “sexy”: conectar estos segmentos a un **dashboard** (Power BI/Looker Studio) para que negocio juegue con los grupos.



## 9) Exporto resultados útiles
Quiero dejar dos archivos listos:  
- Un CSV con el **resumen por cluster**.  
- Un CSV con cada **cliente y su cluster** para que negocio lo pueda cruzar con campañas.


In [ ]:

# Guardo dataset etiquetado
merged = df.copy()
# Si por operaciones de limpieza hubo filtrado de filas, sincronizo índices cuando es posible
if len(X_clustered) == len(merged):
    merged = merged.loc[X_clustered.index]

merged["cluster"] = X_clustered["cluster"].values
out_path = "reports/customers_with_cluster.csv"
merged.to_csv(out_path, index=False)
print("Guardado:", out_path)
